In [1]:
%load_ext autoreload
%autoreload 2
# https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a
# https://github.com/WillKoehrsen/hyperparameter-optimization/blob/master/Bayesian%20Hyperparameter%20Optimization%20of%20Gradient%20Boosting%20Machine.ipynb
from hyperopt import hp
import numpy as np
import lightgbm as lgb
from hyperopt import STATUS_OK
import hyperopt 
from hyperopt import tpe
from hyperopt import Trials
import csv
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer


In [219]:
                                              # Define the search space

space = {
            'class_weight': hp.choice('class_weight', [None, 'balanced']),
            'boosting_type': hp.choice('boosting_type', 
                               [{'boosting_type': 'gbdt', 
                                    'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                 {'boosting_type': 'dart', 
                                     'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                 {'boosting_type': 'goss'}]),
            'num_leaves': hp.choice('num_leaves', np.arange(30, 151, dtype=int)),
            'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.1)),
            'subsample_for_bin': hp.choice('subsample_for_bin', np.arange(1, 16, dtype=int)),
            'min_child_samples': hp.choice('min_child_examples', np.arange(4, 101, dtype=int)),
            'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
            'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
            'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
            'seed':99999,
            'bagging_freq':hp.choice('bagging_freq', 
                               [
                                   {'gbdt': hp.choice('gbdt_bagging_freq', np.arange(1, 25, dtype=int))}, 
                               ]),
            'bagging_fraction':hp.choice('bagging_fraction', 
                               [
                                    {'gbdt': hp.uniform('gbdt_bagging_fraction', 0.95, 1)}, 
                               ]),
            'boost_from_average':'false',
            'feature_fraction':hp.uniform('feature_fraction', 0.05, 1),
            'max_depth':-1,
            'metric':'root_mean_squared_error',
            'min_data_in_leaf':hp.choice('min_data_in_leaf', np.arange(5, 20, dtype=int)),
            'num_threads':1,
            'tree_learner':'serial',
            'objective':'regression',
            'use_missing':True,
            'verbosity':1,

}

 
# Sample from the full space
example = hyperopt.pyll.stochastic.sample(space)

# Dictionary get method with default
subsample = example['boosting_type'].get('subsample', 1.0)

# Assign top-level keys
example['boosting_type'] = example['boosting_type']['boosting_type']
example['bagging_fraction'] = example['bagging_fraction'].get(example['boosting_type'], 1) # default to 1 unless lgtb
example['bagging_freq'] = example['bagging_freq'].get(example['boosting_type'], 0) # default to 0 unless lgtb
example['subsample'] = subsample
example



{'bagging_fraction': 1,
 'bagging_freq': 0,
 'boost_from_average': 'false',
 'boosting_type': 'dart',
 'class_weight': 'balanced',
 'colsample_bytree': 0.8289740768038011,
 'feature_fraction': 0.11610678946916064,
 'learning_rate': 0.012594453789711607,
 'max_depth': -1,
 'metric': 'root_mean_squared_error',
 'min_child_samples': 94,
 'min_data_in_leaf': 5,
 'num_leaves': 49,
 'num_threads': 1,
 'objective': 'regression',
 'reg_alpha': 0.532318489229641,
 'reg_lambda': 0.42263086575159026,
 'seed': 99999,
 'subsample_for_bin': 10,
 'tree_learner': 'serial',
 'use_missing': True,
 'verbosity': 1,
 'subsample': 0.9424891735356111}

In [209]:
example = hyperopt.pyll.stochastic.sample(space)
example['bagging_freq'],example['bagging_freq'].get(example['boosting_type']['boosting_type'], 99), example['boosting_type']['boosting_type']
example['bagging_fraction'] = example['bagging_fraction'].get(example['boosting_type']['boosting_type'], 1) 
example['bagging_fraction'] 

1

In [208]:
example = hyperopt.pyll.stochastic.sample(space)

# Assign top-level keys
example['boosting_type'] = example['boosting_type']['boosting_type']
#example['bagging_fraction'] = example['bagging_fraction'].get(example['boosting_type']['boosting_type'], 1) # default to 1 unless lgtb
example['bagging_freq'] = example['bagging_freq'].get(example['boosting_type']['boosting_type'], 0) # default to 0 unless lgtb



TypeError: string indices must be integers

In [3]:
from baseline5_for_sweep import perform_cv_lightgbm, get_training_set

def objective(params, reporter):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Retrieve the subsample if present otherwise set to 1.0
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['bagging_fraction'] = params['bagging_fraction'].get(params['boosting_type'], 1) # default to 1 unless lgtb
    params['bagging_freq'] = params['bagging_freq'].get(params['boosting_type'], 0) # default to 0 unless lgtb
    params['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    if params['boosting_type'] == 'dart':
        early_stopping_rounds = 0
    else:
        early_stopping_rounds = 100
    
    # Perform n_folds cross validation
    cv_results = perform_cv_lightgbm(df_train, 
                                     chosen_features,
                                     params=params,
                                     early_stopping_rounds=early_stopping_rounds,
                                     n_folds=5) 
    # lgb.cv(params, train_set, num_boost_round = 10000, nfold = n_folds, 
    #                    early_stopping_rounds = 100, metrics = 'auc', seed = 50)
    
    run_time = timer() - start
    
    # Loss must be minimized
    loss = cv_results['rmse_mean']
    
    # Write to the csv file ('a' means append)
    #of_connection = open(out_file, 'a')
    #writer = csv.writer(of_connection)
    #writer.writerow([loss, params, ITERATION, 
    #                 cv_results['rmse_std'], 
    #                 cv_results['cv_stopping_iters'],
    #                 cv_results['best_stopping_iter_std'],
    #                 cv_results['best_stopping_iter_std'],
    #                 run_time])
    
    # Ray will negate this by itself to feed into HyperOpt
    reporter(timesteps_total=1,  mean_loss=cv_results['rmse_std'])

    
    # Dictionary with information for evaluation
    return {'loss': loss,
            'params': params, 
            'iteration': ITERATION,
            'rmse_std': cv_results['rmse_std'],  
            'best_stopping_iter':cv_results['cv_stopping_iters'],
            'best_stopping_iter_mean':cv_results['best_stopping_iter_std'],
            'best_stopping_iter_std':cv_results['best_stopping_iter_std'],
            'train_time': run_time, 
            'status': STATUS_OK}

/media/DATA/anaconda3/envs/thriai/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
df_train, chosen_features = get_training_set()

There is 0.24038838760298156 ratio is nan album
There is 0.0017653981953707335 ratio is nan genre
There is 0.24038838760298156 ratio is nan album_artist
There is 0.0007846214201647705 ratio is nan track
There is 0.6722244017261672 ratio is nan lyric
2711 raw titles are identical between songs: 7485 unique titles
After cleaning brackets etc. only 6202 unique titles remain, i.e. 1283 are highly similar titles 
In making albumHashandName, we filled in: 146 values Of the total albumHashAndName 171 nan
In making albumHashAndNameAndReleaseday, we filled in the: 171 values remaining using the release second hash
There is a statistically signficiant relationship between English-like title and rank. So adding feature: isEnglishLikeTitle
Len before:  10190
Len after:  10183
Missing artist_count_id 799
Missing artist_mean_id 799
Missing artist_std_id 799


In [288]:

from hyperopt import fmin

# Trials object to track progress
bayes_trials = Trials()
# Algorithm
tpe_algorithm = tpe.suggest
# File to save first results
out_file = 'gbm_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)
# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'rmse_std', 'best_stopping_iter',
                 'best_stopping_iter_mean', 'best_stopping_iter_std', 'train_time', 'status'])
    
of_connection.close()

# Global variable
global  ITERATION

ITERATION = 0

MAX_EVALS = 10

# Optimize
best = fmin(fn = objective, 
            space = space, 
            algo = tpe.suggest, 
            max_evals = MAX_EVALS, 
            trials = bayes_trials)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/media/DATA/anaconda3/envs/thriai/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)



Fold 0                                              
  0%|          | 0/10 [00:11<?, ?it/s, best loss: ?]


KeyboardInterrupt: 

In [220]:
import ray
from ray.tune import run
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.logger import DEFAULT_LOGGERS

# Global variable
global  ITERATION

ITERATION = 0

MAX_EVALS = 10
# https://github.com/ray-project/ray/blob/master/python/ray/tune/examples/hyperopt_example.py
config = {
    "num_samples": 2,
    "config": {
        "iterations": 2, # passed to our objective
    },
    "stop": {
        "timesteps_total": 999
    },
}
    
algo = HyperOptSearch(
    space,
    max_concurrent=4,
    metric="mean_loss",
    mode="min")

class TestLogger(ray.tune.logger.Logger):
    def on_result(self, result):
        print("_____________________________________________________________TestLogger", result)

scheduler = AsyncHyperBandScheduler(metric="mean_loss", mode="min")
run(objective,
    name="hyperband_test2",
    search_alg=algo,
    scheduler=scheduler,
    loggers=[ray.tune.logger.CSVLogger, TestLogger],
    **config)


== Status ==
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 0/8 CPUs, 0/1 GPUs, 0.0/1.51 GiB heap, 0.0/0.49 GiB objects
Memory usage on this node: 10.2/15.6 GiB



2019-11-26 00:47:59,911	WARNING worker.py:1619 -- Warning: The actor WrappedFunc has size 17145505 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


== Status ==
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1/8 CPUs, 0/1 GPUs, 0.0/1.51 GiB heap, 0.0/0.49 GiB objects
Memory usage on this node: 10.3/15.6 GiB
Result logdir: /home/ben/ray_results/hyperband_test2
Number of trials: 2 ({'RUNNING': 1, 'PENDING': 1})
PENDING trials:
 - objective_2_gbdt=0.95931,gbdt=6,boost_from_average=false,boosting_type=goss,class_weight=None,colsample_bytree=0.91728,feature_fraction=0.6398,iterations=2,learning_rate=0.022792,max_depth=-1,metric=root_mean_squared_error,min_child_samples=6,min_data_in_leaf=14,num_leaves=136,num_threads=1,objective=regression,reg_alpha=0.444,reg_lambda=0.83056,seed=99999,subsample_for_bin=7,tree_learner=serial,use_missing=True,verbosity=1:	PENDING
RUNNING trials:
 - objective_1_gbdt=0.95012,gbdt=14,boost_from_average=false,boosting_type=gbdt,subsample=0.80368,class_weight=balanced,colsample_bytree=0.76208,feature_fraction=0.2582

2019-11-26 00:48:00,201	WARNING worker.py:1619 -- Warning: The actor WrappedFunc has size 17145505 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


(pid=2889) /media/DATA/anaconda3/envs/thriai/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
(pid=2889)   % (min_groups, self.n_splits)), Warning)
(pid=2889) Fold 0
(pid=4371) /media/DATA/anaconda3/envs/thriai/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
(pid=4371)   % (min_groups, self.n_splits)), Warning)
(pid=4371) Fold 0
(pid=2889) [LightGBM] [Warning] Unknown parameter: class_weight
(pid=2889) [LightGBM] [Warning] Unknown parameter: iterations
(pid=2889) [LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=18 will be ignored. Current value: min_data_in_leaf=8
(pid=2889) [LightGBM] [Warning] feature_fraction is set=0.25823166550

2019-11-26 00:48:32,202	INFO tune.py:276 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.


(pid=4371) CV RMSE: 6.09730 
Result for objective_2_gbdt=0.95931,gbdt=6,boost_from_average=false,boosting_type=goss,class_weight=None,colsample_bytree=0.91728,feature_fraction=0.6398,iterations=2,learning_rate=0.022792,max_depth=-1,metric=root_mean_squared_error,min_child_samples=6,min_data_in_leaf=14,num_leaves=136,num_threads=1,objective=regression,reg_alpha=0.444,reg_lambda=0.83056,seed=99999,subsample_for_bin=7,tree_learner=serial,use_missing=True,verbosity=1:
  date: 2019-11-26_00-48-32
  done: false
  experiment_id: 5e86c963483440ce83b96e6e1faa2f03
  hostname: ben-asus
  iterations_since_restore: 1
  mean_loss: 0.003641263797063363
  neg_mean_loss: -0.003641263797063363
  node_ip: 192.168.0.15
  pid: 4371
  time_since_restore: 29.36853313446045
  time_this_iter_s: 29.36853313446045
  time_total_s: 29.36853313446045
  timestamp: 1574696912
  timesteps_since_restore: 0
  timesteps_total: 1
  training_iteration: 1
  trial_id: f4dd5ede
  
_____________________________________________

(pid=4371) Mean val RMSE 6.0973 +/- 0.0036


In [223]:
DEFAULT_LOGGERS

(ray.tune.logger.JsonLogger,
 ray.tune.logger.CSVLogger,
 <function ray.tune.logger.tf2_compat_logger(config, logdir, trial=None)>)